In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os 
os.chdir('/content/drive/My Drive/Colab_Notebooks/DeepLearningProjects/KerasWordLevelGeneration/JasonBrownlee')

In [1]:
import os
import string
import pickle
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, Dense, Input, LSTM
from keras.models import Model

Using TensorFlow backend.


In [2]:
with open('republic.txt', mode='r') as file:
    print(len(file.readlines()))

15802


### Data Prepration


* Book/Chapter headings (e.g. “BOOK I.”).
* British English spelling (e.g. “honoured”)
* Lots of punctuation (e.g. “–“, “;–“, “?–“, and more)
* Strange names (e.g. “Polemarchus”).
* Some long monologues that go on for hundreds of lines.
* Some quoted dialog (e.g. ‘…’)


In [3]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load document
in_filename = 'republic.txt'
doc = load_doc(in_filename)
print(doc[:200])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what


### Clean Text


* Replace ‘–‘ with a white space so we can split words better.
* Split words based on white space.
* Remove all punctuation from words to reduce the vocabulary size (e.g. ‘What?’ becomes ‘What’).
* Remove all words that are not alphabetic to remove standalone punctuation tokens.
* Normalize all words to lowercase to reduce the vocabulary size.


In [4]:
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [5]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

In [6]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 118633


In [7]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [8]:
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

# Train Language Model

* It uses a distributed representation for words so that different words with similar meanings will have a similar representation.
* It learns the representation at the same time as learning the model.
* It learns to predict the probability for the next word using the context of the last 100 words.


In [9]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [11]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('vocab size is: ', vocab_size)

vocab size is:  7410


In [12]:
# separate into input and output
sequences = np.array(sequences)
x_train, y_train = sequences[:, :-1], sequences[:, -1]
y_train = to_categorical(y_train, num_classes = vocab_size)
seq_length = x_train.shape[1]
print('sequence length is: ', seq_length)
print('x_train shape', x_train.shape)
print('y_train shape', y_train.shape)

sequence length is:  50
x_train shape (118633, 50)
y_train shape (118633, 7410)


In [13]:
# define model
inputs = Input(shape=(50,) ,name='inputs')
x = inputs
x = Embedding(input_dim= vocab_size, output_dim=50)(inputs)
x = LSTM(128, return_sequences= True)(x)
x= LSTM(128)(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(vocab_size, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0902 09:59:37.514068  5444 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 09:59:37.529200  5444 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 09:59:37.530328  5444 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0902 09:59:37.931329  5444 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0902 09:59:37.954182  5444 deprecation_

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            370500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 7410)              955890    
Total params: 1,566,134
Trainable params: 1,566,134
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(x_train, y_train, batch_size=128, epochs=200)

W0831 22:26:33.004870  2896 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0831 22:26:33.100643  2896 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0831 22:26:33.208351  2896 deprecation.py:323] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0831 22:26:34.516002  2896 deprecation_wrapper.py:119] From C:\Users\10\Anaconda3\envs\gpu_env\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_a

Epoch 1/200
118633/118633 [==============================] - 145s 1ms/step - loss: 6.1330 - acc: 0.0741
Epoch 2/200
118633/118633 [==============================] - 142s 1ms/step - loss: 5.6987 - acc: 0.1070
Epoch 3/200
118633/118633 [==============================] - 140s 1ms/step - loss: 5.4531 - acc: 0.1353
Epoch 4/200
118633/118633 [==============================] - 136s 1ms/step - loss: 5.2907 - acc: 0.1489
Epoch 5/200
118633/118633 [==============================] - 136s 1ms/step - loss: 5.2162 - acc: 0.1532
Epoch 6/200
118633/118633 [==============================] - 135s 1ms/step - loss: 5.0960 - acc: 0.1619
Epoch 7/200
118633/118633 [==============================] - 135s 1ms/step - loss: 4.9977 - acc: 0.1673
Epoch 8/200
118633/118633 [==============================] - 135s 1ms/step - loss: 4.9085 - acc: 0.1728
Epoch 9/200
118633/118633 [==============================] - 137s 1ms/step - loss: 4.8246 - acc: 0.1765
Epoch 10/200
118633/118633 [==============================] - 13

118633/118633 [==============================] - 132s 1ms/step - loss: 2.4499 - acc: 0.4402
Epoch 80/200
118633/118633 [==============================] - 132s 1ms/step - loss: 2.4655 - acc: 0.4392
Epoch 81/200
118633/118633 [==============================] - 132s 1ms/step - loss: 2.4163 - acc: 0.4489
Epoch 82/200
118633/118633 [==============================] - 132s 1ms/step - loss: 2.3931 - acc: 0.4539
Epoch 83/200
118633/118633 [==============================] - 132s 1ms/step - loss: 2.3616 - acc: 0.4574
Epoch 84/200
118633/118633 [==============================] - 132s 1ms/step - loss: 2.3475 - acc: 0.4626
Epoch 85/200
118633/118633 [==============================] - 133s 1ms/step - loss: 2.3249 - acc: 0.4669
Epoch 86/200
118633/118633 [==============================] - 132s 1ms/step - loss: 2.3350 - acc: 0.4671
Epoch 87/200
118633/118633 [==============================] - 132s 1ms/step - loss: 2.2765 - acc: 0.4762
Epoch 88/200
118633/118633 [==============================] - 132s 1

In [15]:
# save the model to file
model.save('model.h5')
# save the tokenizer
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

# Use Language Model

In [18]:
# select a seed text
seed_text = lines[np.random.randint(0, len(lines))]
print(seed_text + '\n')

here are three beds one existing in nature which is made by god as i think that we may say for no one else can be the maker no there is another which is the work of the carpenter yes and the work of the painter is a third yes beds



In [28]:
encoded = np.array(tokenizer.texts_to_sequences([seed_text]))
# predict the probabilities for each word
y_hat = np.argmax(model.predict(encoded[:,:seq_length], verbose=0)[0])
out_word = ''
for word, index in tokenizer.word_index.items():
    if index == y_hat:
        out_word = word
        break
print('predicted word is: ', out_word)

(1, 51)

In [49]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = np.array(tokenizer.texts_to_sequences([in_text]))
        encoded = pad_sequences(encoded, maxlen=seq_length, truncating='pre')
        y_hat = np.argmax(model.predict(encoded, verbose=0)[0])
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_hat:
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)
        

In [51]:
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(seed_text)
print(generated)

here are three beds one existing in nature which is made by god as i think that we may say for no one else can be the maker no there is another which is the work of the carpenter yes and the work of the painter is a third yes beds
then are of man as in what men is not as a man to hold in the first place that he has no cares appetites and then they are prescribing because the lovers of natures have a faculty of poets he is the only reason yes he replied then has


### Use Pretrained Embedding

*:)*